In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits import mplot3d
from collections import Counter
import math

In [8]:
root = "/home/subhash/Desktop/spam/nnfl assignment"
data = pd.read_excel(os.path.join(root,'data3.xlsx'),header=None)

In [9]:
def normalize(data):
    for i in range(0,data.shape[1]-1):
        arr = data.iloc[:,i].to_numpy()
        data.iloc[:,i] = (arr - np.mean(arr))/np.std(arr)
    return data

In [10]:
data = normalize(data)
data = data.to_numpy()

In [11]:
def test_train_split(data,test_ratio = 0.4):
    splits = np.split(data[:, :], np.cumsum(np.unique(data[:, 4], return_counts=True)[1])[:-1])
    splits[0][:,4] = np.zeros((1,50))
    splits[1][:,4] = np.ones((1,50))
    train_data = np.array((splits[0][:int(len(splits[0])*(1-test_ratio)),:],splits[1][:int(len(splits[1])*(1-test_ratio)),:]))
    test_data = np.array((splits[0][int(len(splits[0])*(1-test_ratio)):,:],splits[1][int(len(splits[1])*(1-test_ratio)):,:]))
    np.random.shuffle(train_data)
    np.random.shuffle(test_data)
    return train_data,test_data

In [12]:
train_data,test_data = test_train_split(data)
print(train_data.shape)
print(test_data.shape)

(2, 30, 5)
(2, 20, 5)


In [13]:
a1 = [np.mean(train_data[0][:,i]) for i in range(0,4)]
a2 = [np.mean(train_data[1][:,i]) for i in range(0,4)]

e1 = np.dot(np.transpose(train_data[0][:,:4]),train_data[0][:,:4])
e2 = np.dot(np.transpose(train_data[1][:,:4]),train_data[1][:,:4])
det_e1 = np.linalg.det(e1)
det_e2 = np.linalg.det(e2)

p1 = len(train_data[0])/(len(train_data[0])+len(train_data[1]))
p2 = len(train_data[1])/(len(train_data[0])+len(train_data[1]))

In [14]:
def LRT(train_data,test_data,p1,p2,a1,a2,e1,e2,det_e1,det_e2):

    test_data = np.concatenate((test_data[0],test_data[1]),axis=0)
    np.random.shuffle(test_data)
    y_pred = np.zeros((40,1))

    for i in range(len(test_data)):
        x = test_data[i][0:4]
        p_x1 = 1/(((2*math.pi)**0.5)*(det_e1))*np.exp(np.dot(np.transpose(-0.5*(x-a1)),np.dot(np.linalg.pinv(e1),(x-a1))))
        p_x2 = 1/(((2*math.pi)**0.5)*(det_e1))*np.exp(np.dot(np.transpose(-0.5*(x-a2)),np.dot(np.linalg.pinv(e2),(x-a2))))
        p_x = [p_x1/p_x2,p1/p2]
        y_pred[i,0] = np.argmax(p_x)

    return y_pred,test_data

In [16]:
y_pred,test_data = LRT(train_data,test_data,p1,p2,a1,a2,e1,e2,det_e1,det_e2)

In [17]:
def test(y_pred,y_test):
    y_test = y_test.reshape((40,1))
    error = (y_test-np.rint(y_pred)).tolist()
    sum = (y_test+np.rint(y_pred)).tolist()
    FN = error.count([1.0])
    FP = error.count([-1.0])
    TN = sum.count([0.0])
    TP = sum.count([2.0])
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    sensitivity = TP/(TP+FN)
    specificity = TN/(FP+TN)
    confusion_matrix = np.array([[TN,FP],[FN,TP]])
    return confusion_matrix,accuracy, sensitivity, specificity

In [18]:
confusion_matrix,accuracy, sensitivity, specificity = test(y_pred,test_data[:,4])

In [19]:
print("Test accuracy: ",accuracy)
print("Test sensitivity: ",sensitivity)
print("Test specificity: ",specificity)
print("Confusion matrix: ")
print(confusion_matrix)

Test accuracy:  0.125
Test sensitivity:  0.0
Test specificity:  0.25
Confusion matrix: 
[[ 5 15]
 [20  0]]
